# YOLO v8 - hyperparameter tuning

Sources:
- https://github.com/ultralytics/ultralytics
- https://docs.ultralytics.com/guides/hyperparameter-tuning/#best_hyperparametersyaml

Hyperparameter tuning is used for optimizing the model's performance. It is aimed at optimizing metrics such as accuracy, precision, and recall. In the context of Ultralytics YOLO, these hyperparameters could range from learning rate to architectural details (number of layers or types of activation functions used).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

import os
import glob as glob
import matplotlib.pyplot as plt
import random
import cv2
import torch


In [2]:
# Check the files in current directory & delete them if neccessary
!ls

# # remove directories and files
# !rm -rf ./garbage
# !rm -rf ./garbage_sub
# !rm -rf ./runs

# !rm -rf ./yolov8n.pt

# !rm -rf ./fetch_data.sh
# !rm -rf ./requirements.txt

sample_data


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

In [5]:
# Install the ultralytics package
!pip install ultralytics -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 3.8 MB/s eta 0:00:00


In [6]:
from ultralytics import YOLO

### Download the data
- 10% subset of the garbage data in the yolo format.

In [7]:
!wget -O fetch_data.sh https://raw.githubusercontent.com/aml-2023/final-project/main/fetch_data.sh
!bash fetch_data.sh --type yolo --output garbage --percentage subset

--2023-12-22 14:28:14--  https://raw.githubusercontent.com/aml-2023/final-project/main/fetch_data.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2386 (2.3K) [text/plain]
Saving to: ‘fetch_data.sh’

fetch_data.sh       100%[===================>]   2.33K  --.-KB/s    in 0s      

2023-12-22 14:28:14 (36.5 MB/s) - ‘fetch_data.sh’ saved [2386/2386]

yolo dataset
Subset dataset
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27.5M  100 27.5M    0     0  10.2M      0  0:00:02  0:00:02 --:--:-- 10.2M
Data downloaded and extracted into garbage


We use the model.tune() method for hyperparameter tuning. We have set the epochs to 25, the iterations to 10, and the optimizer to AdamW. We skip plotting, checkpointing, and validation other than on the final epoch for faster Tuning.

In [8]:
# Initialize the YOLO model
model = YOLO('yolov8n.pt')

# Tune hyperparameters on the subset
model.tune(data="/content/garbage/data.yaml", epochs=25, iterations=10, optimizer='AdamW', plots=False, save=False, val=False)

100%|██████████| 6.23M/6.23M [00:00<00:00, 67.2MB/s]

Tuner: Initialized Tuner instance with 'tune_dir=runs/detect/tune'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}


Saved runs/detect/tune/tune_scatter_plots.png
Saved runs/detect/tune/tune_fitness.png

Tuner: 1/10 iterations complete ✅ (182.98s)
Tuner: Results saved to runs/detect/tune
Tuner: Best fitness=0.25555 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.69977, 'metrics/recall(B)': 0.51724, 'metrics/mAP50(B)': 0.56408, 'metrics/mAP50-95(B)': 0.22127, 'val/box_loss': 2.04476, 'val/cls_loss': 1.73557, 'val/dfl_loss': 2.91694, 'fitness': 0.25555}
Tuner: Best fitness model is runs/detect/train
Tuner: Best fitness hyperparameters are printed below.

Printing 'runs/detect/tune/best_hyperparameters.yaml'

lr0: 0.01
lrf: 0.01
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
box: 7.5
cls: 0.5
dfl: 1.5
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 0.0
translate: 0.1
scale: 0.5
shear: 0.0
perspective: 0.0
flipud: 0.0
fliplr: 0.5
mosaic: 1.0
mixup: 0.0
copy_paste: 0.0

Tuner: Starting iteration 2/10 with hyperparameters: {'lr0': 0.01043, 'lrf':

From the results above, we can see that the best fitness (0.30269) was observed during the 4th iteration.

We can also see the optimal parameters.

**Best fitness metrics:**
- precision: 0.81491
- recall: 0.51724
- mAP50: 0.73029
- mAP50-95: 0.25548
- val/box_loss: 2.2019
- val/cls_loss: 1.78132
- val/dfl_loss: 2.88193
- fitness: 0.30296



After we complete the hyperparameter tuning, we get several files:
- **best_hyperparameters.yaml** --> A YAML file that contains the optimal hyperparameters that we can later use to initialize future trainings.
- **best_fitness.png** --> A plot displaying fitness (typically a performance metric like AP50) against the number of iterations.
- **tune_results.csv** --> A CSV file containing the results (metrics) of each iteration.
- **tune_scatter_plots.png** --> A file that contains scatter plots generated from tune_results.csv, which can help visualize the relationships between the different hyperparameters and performance metrics.
- **weights/** --> A directory thatcontains the saved PyTorch models for the last and the best iterations during the hyperparameter tuning proces**
